# <h1 style='font-size:2.2rem;color:orange;'>Stock Markets Sensitivity to Social vs. News Media Sentiment</h1>

## 1. Install packages

In [2]:
#pip install transformers
#pip install yfinance

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import bs4 as bs
from selenium import webdriver
import requests
import time
import concurrent.futures
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import yfinance as yf
import os
import json
import os,sys
import string
import itertools
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.stem import WordNetLemmatizer
from collections import Counter
import fastparquet
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single
from afinn import Afinn
from sklearn.preprocessing import MinMaxScaler

## 3. Download datasets | Stock Data | News Data | Twitter Data

### 3.1 Scrap Stock data

In [ ]:
df = pd.DataFrame()
tickers= ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'ATVI', 'NVDA', 'FB', 'UBER', 'V', 'MA', 'AVGO', 'CSCO', 'ADBE', 'CRM', 'AMD', 'INTC', 'NFLX']

for ticker in tickers:
#Get the data for the stock each stock
    data = yf.download(ticker,'2021-09-01','2022-03-01')
    data['Ticker'] = ticker
    data.to_csv('Price_'+ str(ticker) + '.csv')

#Crystal

### 3.2 Scrap News

In [ ]:
#Funtion to scrape news content, from the news links
def linkScraper(newsLinks):
    try:
        linkResp = requests.get(newsLinks)
        linkSoup = bs.BeautifulSoup(linkResp.text, 'lxml')
        
    except:
        print(newsLinks) 
    try:
        url.append(linkSoup.find('meta',{"property":"og:url"}).get('content'))
    except:
        url.append(np.nan)
    try:
        title.append(linkSoup.find('h1',{"class":"text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_2__1K_hh heading__base__2T28j heading__heading_2__3Fcw5"}).text)
    except:
        title.append(np.nan)
    try:        
        date.append(linkSoup.find('span',{"class":"date-line__date__23Ge-"}).text + ' ' + linkSoup.findAll('span',{"class":"date-line__date__23Ge-"})[1].text)
    except:
        date.append(np.nan)
    try:
        author.append(linkSoup.find('a',{"class":"author-name__author__1gx5k"}).text)
    except:
        author.append(np.nan)
    try:
        content.append(linkSoup.find('div',{"class":"article-body__content__3VtU3 paywall-article"}).get_text(separator=' '))
    except:
        content.append(np.nan)
    

#Function to scrape all news links
def newScraper():
    symbols = ['Amazon', 'Apple', 'Microsoft', 'Tesla', 'Blizzard', 'Nvidia', 'Facebook',
               'Uber', 'Mastercard', 'AMD', 'Intel', 'Netflix']
    
    global news, stock, url, title, date, author, content
    stock = []
    url = []
    title = []   
    date = []
    author = []  
    content = [] 
    
    driver = webdriver.Chrome("chromedriver.exe")
    
    for symbol in symbols: 
        searchFormat = 'https://www.reuters.com/site-search/?query={}&offset={}&sort=newest&date=past_year'  
        newsLinks = []

        for i in range(1,180):
            try: 
                driver.get(searchFormat.format(symbol, i*10-10))
                driver.implicitly_wait(3)
                time.sleep(1) 
                
            except:
                continue
            
            for attempt in range(3):
                try: 
                    elems = driver.find_elements_by_css_selector(".search-results__item__2oqiX [href]")
                    if elems != []:
                        for elem in elems:
                            newsLinks.append(elem.get_attribute('href'))

                    else:
                        print('no elements, go to next stock')
                        break 
                    
                except: 
                    print ('webpage error, retrying again')
                    continue

            else:
                continue
            
            break
    
 

        newsLinks = list(set(newsLinks))

        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor: 
            executor.map(linkScraper, newsLinks)
        stock.extend([symbol] * len(newsLinks))
        
        print(str(symbol) + " stock length is : " + str(len(stock)))
        print(str(symbol) + " url length is : " + str(len(url)))
        print(str(symbol) + " title length is : " + str(len(title)))
        print(str(symbol) + " date length is : " + str(len(date)))
        print(str(symbol) + " author length is : " + str(len(author)))
        print(str(symbol) + " content length is : " + str(len(content)))
    
    news = pd.DataFrame({'Stock': stock, 'Date':date, 'Title': title, 'Author': author, 'Content':content, 'Url':url})  
    
    return (news)



In [7]:
news.head(5)

,Stock,Date,Title,Author,Content,Url
0,AMD,"July 2, 2021 9:31 PM UTC",AMD directors dodge shareholder derivative sui...,Jody Godoy,Summary Related documents Shareholder showed n...,https://www.reuters.com/legal/litigation/amd-d...
1,AMD,"April 27, 2021 8:27 PM UTC","AMD lifts revenue forecast, CEO says supply ch...",Reuters,April 27 (Reuters) - Advanced Micro Devices In...,https://www.reuters.com/technology/amd-lifts-a...
2,AMD,"May 25, 2021 1:00 PM UTC",Oracle launches Arm-based cloud computing serv...,Stephen Nellis,May 25 (Reuters) - Oracle Corp (ORCL.N) on T...,https://www.reuters.com/technology/oracle-laun...
3,AMD,"November 29, 2021 8:18 PM UTC",Wall Street regains some ground after virus pu...,Ambar Warrick,"Summary Companies (Updates prices, adds commen...",https://www.reuters.com/markets/europe/wall-st...
4,AMD,"September 21, 2021 6:02 AM UTC",Novartis buys gene therapy firm Arctos Medical...,Reuters,"ZURICH, Sept 21 (Reuters) - Swiss drugmaker No...",https://www.reuters.com/business/healthcare-ph...


### 3.3 Scrap Twitter

In [ ]:
def PreprocessTweet(tweet):
    # process the tweets

    #Remove www.* or https?://*
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    #Remove @username 
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Remove newlines
    tweet = tweet.strip('\n')
    return tweet
  #end
  
#----------------------------------------------------------------------------------

tickers = ['AMZN', 'AAPL', 'MSFT', 'TSLA', 'ATVI', 'NVDA', 'FB', 'UBER', 'MA', 'AMD', 'INTC', 'NFLX']
for i in tickers:
# Using OS library to call CLI commands in Python
    search_term = '#{ticker} ${ticker}'.format(ticker = i)
    snscrape_code = "snscrape --jsonl --since 2021-09-01 twitter-search \"{x} until:2022-03-01\" > {ticker}_tweets.json".format(x = search_term, ticker = i)
    os.system(snscrape_code)

# Reads the json generated from the CLI commands above and creates a pandas dataframe
    read_path = '{ticker}_tweets.json'.format(ticker = i)
    raw_tweets = pd.read_json(read_path, lines=True)

# Filter English tweets
    raw_tweets = raw_tweets[raw_tweets.lang == 'en']
    raw_tweets = raw_tweets.rename(columns={'id': 'tweet_id'})
    
# Filter useful columns
    raw_tweets = raw_tweets[['date','tweet_id','content','user','replyCount','retweetCount','likeCount','quoteCount','hashtags']]
    
# Fetch user column dictionary data
    df = pd.concat([raw_tweets.drop(['user'], axis=1), raw_tweets['user'].apply(pd.Series)], axis=1)
    df = df[['date','tweet_id','content','replyCount','retweetCount','likeCount','quoteCount', 'verified','followersCount', 'friendsCount', 'listedCount', 'mediaCount']]
    df['ticker'] = i

# Remove duplicated tweets to avoid bot tweets
    df = df.drop_duplicates(subset=['content'], keep='first')

#Run pre-processing function
    df['content'] = df['content'].apply(PreprocessTweet)
    
# Output to csv
    df.to_csv(os.path.join('tweet_{ticker}.csv'.format(ticker = i)), index=False)
    print('~~ Finished outputting: ' + 'tweet_{ticker}.csv'.format(ticker = i))


In [9]:
df.head()

,tweet_id,content,replyCount,retweetCount,likeCount,quoteCount,verified,followersCount,friendsCount,listedCount,mediaCount,ticker
date,,,,,,,,,,,,
2022-02-28 07:00:22+00:00,1498191328072675331,$NFLX Weekly. #NFLX 2 hammer candlesticks (one...,0,0,4,0,False,18590,494,520,8381,NFLX
2022-02-24 12:10:07+00:00,1496819727708135428,$NFLX whoops #NFLX,0,0,0,0,False,786,58,25,3007,NFLX
2022-02-23 15:15:28+00:00,1496503985951641605,$nflx #nflx + 48% gain already this AM #tradin...,0,0,1,0,False,20,37,0,39,NFLX
2022-02-23 00:01:18+00:00,1496273928502841344,#Netflix $NFLX #NFLX What a horrendous looking...,0,0,0,0,False,524,1054,6,1328,NFLX
2022-02-22 19:25:44+00:00,1496204581268721664,$NFLX - #NFLX chart on,0,0,0,0,False,1842,0,34,10933,NFLX


## 4. Data Pre-processing | Stock Data | News Data | Twitter Data

### 4.1 News Cleaning

In [ ]:
#Drop nan
news = news.dropna()
news['Content_Clean'] = news.Content
news['Content_Clean'] = [i.replace('Register now for FREE unlimited access to Reuters.com Register ', '')
            .replace(' Our Standards:  The Thomson Reuters Trust Principles.', '') for i in news.Content_Clean]

#Lower case
news.Content_Clean = news.Content_Clean.map(lambda x: x.lower())

#Remove symbols
news.Content_Clean = news.Content_Clean.map(lambda x: re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ', x))
news.Content_Clean = news.Content_Clean.map(lambda x: re.sub('@[^\s]+',' ', x))
news.Content_Clean = news.Content_Clean.map(lambda x: re.sub('[,.\'"!+?\-=)(*><&/;:$%]', ' ', x))
news.Content_Clean = news.Content_Clean.map(lambda x: re.sub('[\s]+', ' ', x))



#Insert ticker column for stocks
ticker = ['AMZN', 'AAPL', 'MSFT', 'TSLA', 'ATVI', 'NVDA', 'FB', 'UBER', 'MA', 'AMD', 'INTC', 'NFLX']
stockDict = dict(zip(news.Stock.unique(), ticker))
tickerList = list(news.Stock.map(stockDict))
news.insert(loc = 0, column = 'Ticker', value = tickerList)

#To datetime
news.Date = pd.to_datetime(news['Date'])
news.Date = news.Date.dt.strftime('%d/%m/%Y %H:%M:%S')



#Tokenize
def newsToken(i):
    for i in news.Content_Clean: 
         yield(gensim.utils.simple_preprocess(str(i), deacc=True)) 
 
    
#Remove stopwords
def newsStopword(content):   
    stopWords = stopwords.words('english') 
    stopWords.extend(['reuters', 'january', 'february', 'march', 'april', 'may', 'june', 
                      'july', 'august', 'september', 'october', 'november', 'devember', 
                      'london', 'summary', 'new york','bengaluru', 'america'])
    keepWords = ['up','down','against', 'above','below','off','over','further','no','not','only']
    stopWords = list(set(stopWords) - set(keepWords))
    return [[i for i in simple_preprocess(str(doc)) 
             if i not in stopWords and len(i) >=1 ] for doc in content] 

    
#Lemmatization
def newsLemmatization(content, allowed_postags=['NOUN', 'PROPN', 'VERB', 'ADP', 'ADJ']):  
    nlp = spacy.load("en_core_web_sm")  #Using spaCy library, loading English package here
    texts_out = []
    for i in content: #Operation for each news doc
        doc = nlp(" ".join(i)) #Join the words together first for lemma analysis
        #Get the lemma for each word
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags]) 
    return texts_out


    
#Execute step by step
matrix1 = list(newsToken(news.Content_Clean))  
matrix2 = newsLemmatization(matrix1)
matrix3 = newsStopword(matrix2) 

#Insert columns for tokenized content
news.insert(loc = 7 , column = 'Content_Clean_Token', value = matrix3)

In [18]:
news.head()

,Ticker,Stock,Date,Title,Author,Content,Content_Clean,Content_Clean_Token,Url
0,AMD,AMD,02/07/2021 21:31:00,AMD directors dodge shareholder derivative sui...,Jody Godoy,Summary Related documents Shareholder showed n...,related document shareholder show basis skip d...,['related' 'document' 'shareholder' 'show' 'ba...,https://www.reuters.com/legal/litigation/amd-d...
1,AMD,AMD,27/04/2021 20:27:00,"AMD lifts revenue forecast, CEO says supply ch...",Reuters,April 27 (Reuters) - Advanced Micro Devices In...,advance micro devices inc amd raise annual rev...,['advance' 'micro' 'devices' 'inc' 'amd' 'rais...,https://www.reuters.com/technology/amd-lifts-a...
2,AMD,AMD,25/05/2021 13:00:00,Oracle launches Arm-based cloud computing serv...,Stephen Nellis,May 25 (Reuters) - Oracle Corp (ORCL.N) on T...,oracle corp orcl tuesday launch cloud computin...,['oracle' 'corp' 'orcl' 'tuesday' 'launch' 'cl...,https://www.reuters.com/technology/oracle-laun...
3,AMD,AMD,29/11/2021 20:18:00,Wall Street regains some ground after virus pu...,Ambar Warrick,"Summary Companies (Updates prices, adds commen...",company update price add commentary change byl...,['company' 'update' 'price' 'add' 'commentary'...,https://www.reuters.com/markets/europe/wall-st...
4,AMD,AMD,21/09/2021 06:02:00,Novartis buys gene therapy firm Arctos Medical...,Reuters,"ZURICH, Sept 21 (Reuters) - Swiss drugmaker No...",zurich sept swiss drugmaker novartis novn say ...,['zurich' 'sept' 'swiss' 'drugmaker' 'novartis...,https://www.reuters.com/business/healthcare-ph...


### 4.2 Tweets Cleaning

In [ ]:
#Merge all stock csv
df = pd.DataFrame()
for file in glob.glob("*.csv"):
    df1 = pd.read_csv(file)
    df = pd.concat([df, df1])
    
#To datetime
df.date = pd.to_datetime(df['date'])
df.date = df.date.dt.strftime('%d/%m/%Y %H:%M:%S')

def _1_TextprocessTweet(tweet):
    
    #Convert to lower case
    tweet = tweet.lower()
	#Remove $companytag 
    tweet = re.sub('\$[^\s]+','',tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #remove punctuation
    tweet = tweet.replace('\'','')
    tweet = re.sub('[%s]' % re.escape(string.punctuation), ' ', tweet) 

    return tweet
  #end


#Tokenize
def _2_Tokenize_Stopword(tweet):   
    tokens = [w for w in word_tokenize(tweet) if w.isalpha()] 
    stopWords = stopwords.words('english') 
    keepWords = ['up','down','against', 'above','below','off','over','further','no','not','only']
    stopWords = list(set(stopWords) - set(keepWords))
    stopWords.extend(['amzn', 'aapl', 'msft', 'tsla', 'atvi', 'nvda', 'fb', 'uber', 'ma', 'amd', 'intc', 'nflx'])

    tweet = [t for t in tokens if t not in stopWords]
    return tweet


#Lemmatization
def _3_Lemmatization(tweet, allowed_postags=['NOUN', 'PROPN', 'VERB', 'ADP','ADJ']):  
    nlp = spacy.load("en_core_web_sm")  #Using spaCy library, loading English package here
    doc = nlp(" ".join(tweet))
    tweet = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return tweet


#Populate various processed columns
df['content_clean'] = df['content'].apply(_1_TextprocessTweet)
df['content_tokenized'] = df['content_clean'].apply(_2_Tokenize_Stopword)


#Using concurrency to speed up the lemmatization function
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor: 
    df['content_lemmatized'] = list(executor.map(_3_Lemmatization, df['content_tokenized']))

In [25]:
df.head()

,date,tweet_id,content,replyCount,retweetCount,likeCount,quoteCount,verified,followersCount,friendsCount,listedCount,mediaCount,ticker,content_clean,content_tokenized,content_lemmatized
0,28/02/2022 22:39:16,1498427612665987078,$SPY #SPY #QQQ $QQQ $UVXY #UVXY $AMC #AMC $AAP...,0,1,2,0,False,279,1026,4,833,AAPL,spy qqq uvxy amc aapl gme tsla btcusdt...,"['spy', 'qqq', 'uvxy', 'amc', 'gme', 'btcusdt'...","['spy', 'qqq', 'uvxy', 'amc', 'gme', 'btcusdt'..."
1,28/02/2022 21:42:08,1498413233698443272,$AAPL held Premarket high for a bit in morning...,0,0,0,0,False,60,258,4,32,AAPL,held premarket high for a bit in morning befo...,"['held', 'premarket', 'high', 'bit', 'morning'...","['hold', 'premarket', 'high', 'bit', 'morning'..."
2,28/02/2022 21:30:00,1498410179972399105,$AAPL closed today at $165.12. If you bought 1...,0,0,0,0,False,50,11,0,1,AAPL,closed today at if you bought 1 share of aap...,"['closed', 'today', 'bought', 'share', 'closin...","['close', 'today', 'buy', 'share', 'closing', ..."
3,28/02/2022 10:28:15,1498243644431732743,$SPY #SPY #QQQ $QQQ $AMC #AMC $AAPL #AAPL $GME...,0,1,2,0,False,279,1026,4,833,AAPL,spy qqq amc aapl gme tsla ukrainerussiaw...,"['spy', 'qqq', 'amc', 'gme', 'ukrainerussiawar...","['spy', 'qqq', 'amc', 'gme', 'ukrainerussiawar..."
4,28/02/2022 04:57:58,1498160526311915520,$AAPL Weekly. #AAPL formed a hammer candlestic...,0,0,12,0,False,18591,494,520,8381,AAPL,weekly aapl formed a hammer candlestick last...,"['weekly', 'formed', 'hammer', 'candlestick', ...","['weekly', 'form', 'hammer', 'candlestick', 'l..."


## 5. Sentiment Analysis | News Data | Twitter Data

### 5.1 Library Flair (For news only)

In [ ]:
#Flair is sentence based
#So we use sentence instead of token for this package
classifier = TextClassifier.load('en-sentiment')
news = news.dropna()

def flairClean(row):
    rows = row.replace('Register now for FREE unlimited access to Reuters.com Register ', '').replace(
        ' Our Standards:  The Thomson Reuters Trust Principles.', '') 

    rows = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ', rows)
    rows = re.sub('@[^\s]+',' ', rows)
    rows = re.sub('[\'"!+?=)(*><&/;:$%\n]', ' ', rows)
    rows = re.sub('[\s]+', ' ', rows)
    return rows


def sentence(paragraph):
    sentences = [elm for elm in split_single(paragraph)]
    return sentences


def predict(sentence):
    text = Sentence(sentence)
    classifier.predict(text)
    
    value = text.labels[0].to_dict()['value'] 
    if value == 'POSITIVE':
        result = text.labels[0].to_dict()['confidence']
    else:
        result = -(text.labels[0].to_dict()['confidence'])
    return round(result, 3)


def flair(sentences):
    results = []
    for i in range(0, len(sentences)): 
        results.append(predict(sentences[i]))
    return results



def flairSum(scores):
    numer = round(sum(scores), 3)
    denom = round(sum([abs(elm) for elm in scores]), 3)
    return (numer/denom)


news['Sentiment_Flair'] = news['Content'].apply(flairClean).apply(sentence).apply(flair).apply(flairSum)
#Not applicable for tweets

In [33]:
#del later
news = pd.read_csv(r'F:\HKU\5\FINA4350\Group Project\News Data\News.csv', index_col = 0)
news = news.drop(['Sentiment_afinn'], axis=1)

news.head()

,Ticker,Stock,Date,Title,Author,Content,Content_Clean,Content_Clean_Token,Url,Sentiment_Flair
0,AMD,AMD,02/07/2021 21:31:00,AMD directors dodge shareholder derivative sui...,Jody Godoy,Summary Related documents Shareholder showed n...,related document shareholder show basis skip d...,['related' 'document' 'shareholder' 'show' 'ba...,https://www.reuters.com/legal/litigation/amd-d...,-0.600430
1,AMD,AMD,27/04/2021 20:27:00,"AMD lifts revenue forecast, CEO says supply ch...",Reuters,April 27 (Reuters) - Advanced Micro Devices In...,advance micro devices inc amd raise annual rev...,['advance' 'micro' 'devices' 'inc' 'amd' 'rais...,https://www.reuters.com/technology/amd-lifts-a...,0.370084
2,AMD,AMD,25/05/2021 13:00:00,Oracle launches Arm-based cloud computing serv...,Stephen Nellis,May 25 (Reuters) - Oracle Corp (ORCL.N) on T...,oracle corp orcl tuesday launch cloud computin...,['oracle' 'corp' 'orcl' 'tuesday' 'launch' 'cl...,https://www.reuters.com/technology/oracle-laun...,0.436454
3,AMD,AMD,29/11/2021 20:18:00,Wall Street regains some ground after virus pu...,Ambar Warrick,"Summary Companies (Updates prices, adds commen...",company update price add commentary change byl...,['company' 'update' 'price' 'add' 'commentary'...,https://www.reuters.com/markets/europe/wall-st...,-0.482530
4,AMD,AMD,21/09/2021 06:02:00,Novartis buys gene therapy firm Arctos Medical...,Reuters,"ZURICH, Sept 21 (Reuters) - Swiss drugmaker No...",zurich sept swiss drugmaker novartis novn say ...,['zurich' 'sept' 'swiss' 'drugmaker' 'novartis...,https://www.reuters.com/business/healthcare-ph...,0.250985


### 5.2 Library afinn

In [ ]:
def afinnSum(scores):
    if len(scores) == 0:
        score = 0
    elif len(scores) == 1 and sum(scores) > 0:
        score = 1
    elif len(scores) == 1 and sum(scores) < 0:
        score = -1
    elif sum(scores) == 0:  
        score = 0
    else:
        numer = sum(scores)
        denom = sum([abs(elm) for elm in scores])
        score = (numer/denom)
    return score



def afinn(text):  
    afinn = Afinn(language = 'en')    
    score = [afinn.score(i) for i in text]
    return score
    
    
df['content_lemmatized'].apply(afinn).apply(afinnSum)
news['Content_Clean_Token'].apply(afinn).apply(afinnSum)

In [37]:
#del later
news = pd.read_csv(r'F:\HKU\5\FINA4350\Group Project\News Data\News.csv', index_col = 0)
news.head()

,Ticker,Stock,Date,Title,Author,Content,Content_Clean,Content_Clean_Token,Url,Sentiment_Flair,Sentiment_afinn
0,AMD,AMD,02/07/2021 21:31:00,AMD directors dodge shareholder derivative sui...,Jody Godoy,Summary Related documents Shareholder showed n...,related document shareholder show basis skip d...,['related' 'document' 'shareholder' 'show' 'ba...,https://www.reuters.com/legal/litigation/amd-d...,-0.600430,-0.780220
1,AMD,AMD,27/04/2021 20:27:00,"AMD lifts revenue forecast, CEO says supply ch...",Reuters,April 27 (Reuters) - Advanced Micro Devices In...,advance micro devices inc amd raise annual rev...,['advance' 'micro' 'devices' 'inc' 'amd' 'rais...,https://www.reuters.com/technology/amd-lifts-a...,0.370084,0.705882
2,AMD,AMD,25/05/2021 13:00:00,Oracle launches Arm-based cloud computing serv...,Stephen Nellis,May 25 (Reuters) - Oracle Corp (ORCL.N) on T...,oracle corp orcl tuesday launch cloud computin...,['oracle' 'corp' 'orcl' 'tuesday' 'launch' 'cl...,https://www.reuters.com/technology/oracle-laun...,0.436454,0.882353
3,AMD,AMD,29/11/2021 20:18:00,Wall Street regains some ground after virus pu...,Ambar Warrick,"Summary Companies (Updates prices, adds commen...",company update price add commentary change byl...,['company' 'update' 'price' 'add' 'commentary'...,https://www.reuters.com/markets/europe/wall-st...,-0.482530,0.025641
4,AMD,AMD,21/09/2021 06:02:00,Novartis buys gene therapy firm Arctos Medical...,Reuters,"ZURICH, Sept 21 (Reuters) - Swiss drugmaker No...",zurich sept swiss drugmaker novartis novn say ...,['zurich' 'sept' 'swiss' 'drugmaker' 'novartis...,https://www.reuters.com/business/healthcare-ph...,0.250985,0.047619


In [38]:
df = pd.read_csv(r'F:\HKU\5\FINA4350\Group Project\Tweets Data\allTWEETS_Processed.csv', index_col = 0)
df.head()

,date,tweet_id,content,replyCount,retweetCount,likeCount,quoteCount,verified,followersCount,friendsCount,listedCount,mediaCount,ticker,content_clean,content_tokenized,content_lemmatized,Sentiment_afinn
0,28/02/2022 22:39:16,1498427612665987078,$SPY #SPY #QQQ $QQQ $UVXY #UVXY $AMC #AMC $AAP...,0,1,2,0,False,279,1026,4,833,AAPL,spy qqq uvxy amc aapl gme tsla btcusdt...,"['spy', 'qqq', 'uvxy', 'amc', 'gme', 'btcusdt'...","['spy', 'qqq', 'uvxy', 'amc', 'gme', 'btcusdt'...",-1.0
1,28/02/2022 21:42:08,1498413233698443272,$AAPL held Premarket high for a bit in morning...,0,0,0,0,False,60,258,4,32,AAPL,held premarket high for a bit in morning befo...,"['held', 'premarket', 'high', 'bit', 'morning'...","['hold', 'premarket', 'high', 'bit', 'morning'...",1.0
2,28/02/2022 21:30:00,1498410179972399105,$AAPL closed today at $165.12. If you bought 1...,0,0,0,0,False,50,11,0,1,AAPL,closed today at if you bought 1 share of aap...,"['closed', 'today', 'bought', 'share', 'closin...","['close', 'today', 'buy', 'share', 'closing', ...",1.0
3,28/02/2022 10:28:15,1498243644431732743,$SPY #SPY #QQQ $QQQ $AMC #AMC $AAPL #AAPL $GME...,0,1,2,0,False,279,1026,4,833,AAPL,spy qqq amc aapl gme tsla ukrainerussiaw...,"['spy', 'qqq', 'amc', 'gme', 'ukrainerussiawar...","['spy', 'qqq', 'amc', 'gme', 'ukrainerussiawar...",-1.0
4,28/02/2022 04:57:58,1498160526311915520,$AAPL Weekly. #AAPL formed a hammer candlestic...,0,0,12,0,False,18591,494,520,8381,AAPL,weekly aapl formed a hammer candlestick last...,"['weekly', 'formed', 'hammer', 'candlestick', ...","['weekly', 'form', 'hammer', 'candlestick', 'l...",0.0


### 5.3 Library NLTK

### 5.4 Library TextBlob

### 5.6 Library Finbert